In [2]:
import os
%pwd
os.chdir("../")




In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class model_trainer_config:
    root_dir: Path
    seq_len:int
    num_epochs: int
    model_folder: str
    model_basename:str
    preload: None
    experiment_name: str
    lr:float
    d_model:int

    
    




In [12]:
from src.Mini_Translator_T.constants import *
from src.Mini_Translator_T.utils.common import read_yaml, create_directories


In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_train_model_config(self) -> model_trainer_config:
        config = self.config.model_trainer
        params=self.params

        create_directories([config.root_dir])

        trainer_config=model_trainer_config(root_dir=config.root_dir,seq_len=params.seq_len,num_epochs=params.num_epochs,
                                            model_folder=config.model_folder,model_basename=config.model_basename,
                                            experiment_name=config.experiment_name,lr=params.lr,d_model=params.d_model,preload=None)
        
        return trainer_config


In [23]:
from tokenizers import Tokenizer

In [24]:
import torch.nn as nn
import torch
import random
import tqdm
import numpy as np
import json
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from src.Mini_Translator_T.logging import logger


In [25]:
from src.Mini_Translator_T import (Transformer,InputEmbeddings,PositionalEncoding,
MultiHeadAttentionBlock,FeedForwardBlock,EncoderBlock,DecoderBlock,Encoder,Decoder,ProjectionLayer)

In [26]:
def casual_mask(size):
        # Creating a square matrix of dimensions 'size x size' filled with ones
        mask = torch.triu(torch.ones(1, size, size), diagonal = 1).type(torch.int)
        return mask == 0

In [27]:
# Define function to obtain the most probable next token
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    # Retrieving the indices from the start and end of sequences of the target tokens
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Computing the output of the encoder for the source sequence
    encoder_output = model.encode(source, source_mask)
    # Initializing the decoder input with the Start of Sentence token
    decoder_input = torch.empty(1,1).fill_(sos_idx).type_as(source).to(device)

    # Looping until the 'max_len', maximum length, is reached
    while True:
        if decoder_input.size(1) == max_len:
            break

        # Building a mask for the decoder input
        decoder_mask = casual_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # Calculating the output of the decoder
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # Applying the projection layer to get the probabilities for the next token
        prob = model.project(out[:, -1])

        # Selecting token with the highest probability
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1,1). type_as(source).fill_(next_word.item()).to(device)], dim=1)

        # If the next token is an End of Sentence token, we finish the loop
        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0) # Sequence of tokens generated by the decoder

In [28]:
# Defining function to evaluate the model on the validation dataset
# num_examples = 2, two examples per run
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples=2):
    model.eval() # Setting model to evaluation mode
    count = 0 # Initializing counter to keep track of how many examples have been processed

    console_width = 80 # Fixed witdh for printed messages

    # Creating evaluation loop
    with torch.no_grad(): # Ensuring that no gradients are computed during this process
        for batch in validation_ds:
            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            # Ensuring that the batch_size of the validation set is 1
            assert encoder_input.size(0) ==  1, 'Batch size must be 1 for validation.'

            # Applying the 'greedy_decode' function to get the model's output for the source text of the input batch
            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            # Retrieving source and target texts from the batch
            source_text = batch['src_text'][0]
            target_text = batch['tgt_text'][0] # True translation
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy()) # Decoded, human-readable model output

            # Printing results
            print_msg('-'*console_width)
            print_msg(f'SOURCE: {source_text}')
            print_msg(f'TARGET: {target_text}')
            print_msg(f'PREDICTED: {model_out_text}')

            # After two examples, we break the loop
            if count == num_examples:
                break

In [29]:
class modelTrainer:
    def __init__(self,config:model_trainer_config,config_filepath = CONFIG_FILE_PATH):
        self.config=config
        self.config2=config_filepath


    # Building & Initializing Transformer

    def get_weights_file_path(self,config, epoch: str):
        config=self.config
        model_folder = config.model_folder # Extracting model folder from the config
        model_basename = config.model_basename # Extracting the base name for model files
        model_filename = f"{model_basename}{epoch}.pt" # Building filename
        return str(Path('.')/ model_folder/ model_filename) # Combining current directory, the model folder, and the model filename

# Definin function and its parameter, including model dimension, number of encoder and decoder stacks, heads, etc.
    def build_transformer(self,src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048) -> Transformer:

        # Creating Embedding layers
        src_embed = InputEmbeddings(d_model, src_vocab_size) # Source language (Source Vocabulary to 512-dimensional vectors)
        tgt_embed = InputEmbeddings(d_model, tgt_vocab_size) # Target language (Target Vocabulary to 512-dimensional vectors)

        # Creating Positional Encoding layers
        src_pos = PositionalEncoding(d_model, src_seq_len, dropout) # Positional encoding for the source language embeddings
        tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout) # Positional encoding for the target language embeddings

        # Creating EncoderBlocks
        encoder_blocks = [] # Initial list of empty EncoderBlocks
        for _ in range(N): # Iterating 'N' times to create 'N' EncoderBlocks (N = 6)
            encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # Self-Attention
            feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # FeedForward

            # Combine layers into an EncoderBlock
            encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
            encoder_blocks.append(encoder_block) # Appending EncoderBlock to the list of EncoderBlocks

        # Creating DecoderBlocks
        decoder_blocks = [] # Initial list of empty DecoderBlocks
        for _ in range(N): # Iterating 'N' times to create 'N' DecoderBlocks (N = 6)
            decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # Self-Attention
            decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # Cross-Attention
            feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # FeedForward

            # Combining layers into a DecoderBlock
            decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
            decoder_blocks.append(decoder_block) # Appending DecoderBlock to the list of DecoderBlocks

        # Creating the Encoder and Decoder by using the EncoderBlocks and DecoderBlocks lists
        encoder = Encoder(nn.ModuleList(encoder_blocks))
        decoder = Decoder(nn.ModuleList(decoder_blocks))

        # Creating projection layer
        projection_layer = ProjectionLayer(d_model, tgt_vocab_size) # Map the output of Decoder to the Target Vocabulary Space

        # Creating the transformer by combining everything above
        transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

        # Initialize the parameters
        for p in transformer.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

        return transformer # Assembled and initialized Transformer. Ready to be trained and validated!
    # We pass as parameters the config dictionary, the length of the vocabylary of the source language and the target language
    def get_model(self,config, vocab_src_len, vocab_tgt_len):

        # Loading model using the 'build_transformer' function.
        # We will use the lengths of the source language and target language vocabularies, the 'seq_len', and the dimensionality of the embeddings
        model = self.build_transformer(vocab_src_len, vocab_tgt_len, config.seq_len, config.seq_len, config.d_model)
        return model

# Function to construct the path for saving and retrieving model weights
    def get_weights_file_path(self,config, epoch: str):
        model_folder = config['model_folder'] # Extracting model folder from the config
        model_basename = config['model_basename'] # Extracting the base name for model files
        model_filename = f"{model_basename}{epoch}.pt" # Building filename
        return str(Path('.')/ model_folder/ model_filename) # Combining current directory, the model folder, and the model filename
    

    def initiate_model_trainer(self):
        config=self.config
        root_dir = "artifacts/data_transformation"
        train_data_loader_path = os.path.join(root_dir, "train_data_loader.pth")
        valid_data_loader_path = os.path.join(root_dir, "valid_data_loader.pth")

        
        tokenizer_en = os.path.join(root_dir, "'tokenizer_en.json'")
        tokenizer_it = os.path.join(root_dir, "'tokenizer_it.json'")
        tokenizer_src = Tokenizer.from_file(tokenizer_en)
        tokenizer_tgt = Tokenizer.from_file(tokenizer_it)



        # Load the DataLoader objects
        train_data_loader = torch.load(train_data_loader_path)
        valid_data_loader = torch.load(valid_data_loader_path)
      


        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device {device}")

        # Creating model directory to store weights
        Path(config.model_folder).mkdir(parents=True, exist_ok=True)

        # Retrieving dataloaders and tokenizers for source and target languages using the 'get_ds' function

        # Initializing model on the GPU using the 'get_model' function
        model = self.get_model(config,tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)



            # Tensorboard
        writer = SummaryWriter(config.experiment_name)

        # Setting up the Adam optimizer with the specified learning rate from the '
        # config' dictionary plus an epsilon value
        optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, eps = 1e-9)

        # Initializing epoch and global step variables
        initial_epoch = 0
        global_step = 0

        # Checking if there is a pre-trained model to load
        # If true, loads it
        if config.preload:
            model_filename = self.get_weights_file_path(config, config.preload)
            print(f'Preloading model {model_filename}')
            state = torch.load(model_filename) # Loading model

            # Sets epoch to the saved in the state plus one, to resume from where it stopped
            initial_epoch = state['epoch'] + 1
            # Loading the optimizer state from the saved model
            optimizer.load_state_dict(state['optimizer_state_dict'])
            # Loading the global step state from the saved model
            global_step = state['global_step']

        # Initializing CrossEntropyLoss function for training
        # We ignore padding tokens when computing loss, as they are not relevant for the learning process
        # We also apply label_smoothing to prevent overfitting
        loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id('[PAD]'), label_smoothing = 0.1).to(device)

        # Initializing training loop

        # Iterating over each epoch from the 'initial_epoch' variable up to
        # the number of epochs informed in the config
        for epoch in range(initial_epoch, config.num_epochs):

            # Initializing an iterator over the training dataloader
            # We also use tqdm to display a progress bar
            batch_iterator = tqdm(train_data_loader, desc = f'Processing epoch {epoch:02d}')

            # For each batch...
            for batch in batch_iterator:
                model.train() # Train the model

                # Loading input data and masks onto the GPU
                encoder_input = batch['encoder_input'].to(device)
                decoder_input = batch['decoder_input'].to(device)
                encoder_mask = batch['encoder_mask'].to(device)
                decoder_mask = batch['decoder_mask'].to(device)

                # Running tensors through the Transformer
                encoder_output = model.encode(encoder_input, encoder_mask)
                decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
                proj_output = model.project(decoder_output)

                # Loading the target labels onto the GPU
                label = batch['label'].to(device)

                # Computing loss between model's output and true labels
                loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))

                # Updating progress bar
                batch_iterator.set_postfix({f"loss": f"{loss.item():6.3f}"})

                writer.add_scalar('train loss', loss.item(), global_step)
                writer.flush()

                # Performing backpropagation
                loss.backward()

                # Updating parameters based on the gradients
                optimizer.step()

                # Clearing the gradients to prepare for the next batch
                optimizer.zero_grad()

                global_step += 1 # Updating global step count

            # We run the 'run_validation' function at the end of each epoch
            # to evaluate model performance
            run_validation(model, valid_data_loader, tokenizer_src, tokenizer_tgt, config.seq_len, device, lambda msg: batch_iterator.write(msg), global_step, writer)

            # Saving model
            model_filename = self.get_weights_file_path(config, f'{epoch:02d}')
            # Writting current model state to the 'model_filename'
            torch.save({
                'epoch': epoch, # Current epoch
                'model_state_dict': model.state_dict(),# Current model state
                'optimizer_state_dict': optimizer.state_dict(), # Current optimizer state
                'global_step': global_step # Current global step
            }, model_filename)


In [30]:
try:
    config = ConfigurationManager()
    get_model_config = config.get_train_model_config()
    model = modelTrainer(config=get_model_config)
    model.initiate_model_trainer()
except Exception as e:
    raise e

[2024-05-28 10:30:09,009: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-28 10:30:09,022: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-28 10:30:09,031: INFO: common: created directory at: artifacts]
[2024-05-28 10:30:09,035: INFO: common: created directory at: artifacts/trained_model]


KeyboardInterrupt: 